In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection._validation import _backtesting_forecaster
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(np.__version__)
print(pd.__version__)
import skforecast
print(skforecast.__version__)

1.26.4
2.2.3
0.14.0


In [5]:
expected_metric = pd.DataFrame({"mean_squared_error": [0.11240154357887522]})
expected_predictions = pd.DataFrame(
    {
        "pred": np.array(
            [
                0.37189345, 0.24877906, 0.33164618, 0.49643827, 0.40030688,
                0.42947433, 0.4592766 , 0.55463757, 0.30374867, 0.32671601,
                0.56153282, 0.55311912
            ]
        )
    },
    index=pd.RangeIndex(start=38, stop=50, step=1),
)

window_features = RollingFeatures(
    stats = ['mean', 'min', 'max', 'sum', 'median', 'ratio_min_max'],
    window_sizes = 3,
)
forecaster = ForecasterRecursive(
    regressor=LinearRegression(), lags=3, window_features=window_features
)

n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
        steps                 = 4,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster = forecaster,
                                    y          = y,
                                    exog       = exog,
                                    cv         = cv,
                                    metric     = 'mean_squared_error',
                                    verbose    = False
                                )

pd.testing.assert_frame_equal(expected_metric, metric)
pd.testing.assert_frame_equal(expected_predictions, backtest_predictions)

100%|██████████| 3/3 [00:00<00:00, 249.78it/s]


AssertionError: DataFrame.iloc[:, 0] (column name="mean_squared_error") are different

DataFrame.iloc[:, 0] (column name="mean_squared_error") values are different (100.0 %)
[index]: [0]
[left]:  [0.11240154357887522]
[right]: [0.10712193574869683]
At positional index 0, first diff: 0.11240154357887522 != 0.10712193574869683

In [7]:
backtest_predictions.to_numpy().flatten()

array([0.37683092, 0.27485253, 0.35459493, 0.50933738, 0.40740455,
       0.44359783, 0.47699345, 0.53865623, 0.30365329, 0.34961539,
       0.5671574 , 0.56751209])

In [8]:
expected_metric = pd.DataFrame({"mean_squared_error": [0.11337407]})
expected_predictions = pd.DataFrame(
    {
        "pred": np.array(
            [
                0.31478125, 0.45940314, 0.85744121, 0.67976412, 0.38085263,
                0.56357911, 0.35567212, 0.4493073 , 0.35493806, 0.53589892,
                0.73060039, 0.60025524
            ]
        )
    },
    index=pd.RangeIndex(start=38, stop=50, step=1),
)

window_features = RollingFeatures(
    stats = ['mean', 'min', 'max', 'sum', 'median', 'ratio_min_max'],
    window_sizes = 3,
)
forecaster = ForecasterDirect(
    regressor=LinearRegression(), steps=4, lags=3, window_features=window_features
)

n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
        steps                 = 4,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster = forecaster,
                                    y          = y,
                                    exog       = exog,
                                    cv         = cv,
                                    metric     = 'mean_squared_error',
                                    verbose    = False
                                )

pd.testing.assert_frame_equal(expected_metric, metric)
pd.testing.assert_frame_equal(expected_predictions, backtest_predictions)

100%|██████████| 3/3 [00:00<00:00, 97.46it/s]


AssertionError: DataFrame.iloc[:, 0] (column name="mean_squared_error") are different

DataFrame.iloc[:, 0] (column name="mean_squared_error") values are different (100.0 %)
[index]: [0]
[left]:  [0.11337407]
[right]: [0.10447263857902568]
At positional index 0, first diff: 0.11337407 != 0.10447263857902568

In [9]:
backtest_predictions.to_numpy().flatten()

array([0.32372783, 0.43458024, 0.77238875, 0.60657529, 0.37897279,
       0.542262  , 0.31403908, 0.37007796, 0.35614498, 0.51622723,
       0.67746181, 0.59161821])